# RankNET

In [10]:
# Libaries
import os
import gc
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.nn import leaky_relu
from sklearn.metrics import accuracy_score
from scipy.stats import pearsonr, kendalltau
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import GroupKFold
from tensorflow.keras.layers import Dense, Subtract, Dropout, Input
from tensorflow.keras.models import Model


warnings.simplefilter(action='ignore', category=FutureWarning)

In [11]:
project_directory = r'C:\Users\marco\OneDrive\Desktop\Final Year Project'
os.chdir(project_directory)
base_dir = os.getcwd() 

- Evaluation for Heist! game

In [12]:
base_dir = 'AGAIN Ranking Algorithms'

def pairwise_transformation(X, Y, groups):
    transformed_data, labels, new_groups = [], [], []
    for i in range(len(X)):
        for j in range(i+1, len(X)):  
            if abs(Y[i] - Y[j]) > 0.1:
                transformed_data.append(np.concatenate([X[i], X[j]]))
                labels.append(1 if Y[i] > Y[j] else 0)
                new_groups.append(groups[i])

                transformed_data.append(np.concatenate([X[j], X[i]]))
                labels.append(0 if Y[i] > Y[j] else 1)
                new_groups.append(groups[j])
    return np.array(transformed_data), np.array(labels), np.array(new_groups)

class RankNet(Model):
    def __init__(self, input_shape):
        super(RankNet, self).__init__()
        self.dense1 = Dense(16, activation='relu')
        self.dropout = Dropout(0.1)
        self.dense2 = Dense(8, activation='relu')
        self.output_layer = Dense(1, activation='sigmoid')  
        self.subtract_layer = Subtract()
        
    def call(self, inputs):
        half = inputs.shape[1] // 2
        xi, xj = inputs[:, :half], inputs[:, half:]
        xi_output = self.dense2(self.dropout(self.dense1(xi)))
        xj_output = self.dense2(self.dropout(self.dense1(xj)))
        oi = self.output_layer(xi_output)
        oj = self.output_layer(xj_output)
        oij = self.subtract_layer([oi, oj])
        return oij 

def evaluate_individual_performance(model, X_test, Y_test, group_labels, game_name):
    evaluation_results = []
    for participant_id in np.unique(group_labels):
        idx = group_labels == participant_id
        participant_X, participant_Y = X_test[idx], Y_test[idx]
        predicted_scores = model.predict(participant_X).flatten()
        
        evaluation_results.append({
            'Game': game_name,
            'Participant ID': participant_id,
            'CCC': concordance_correlation_coefficient(participant_Y, predicted_scores),
            'PCC': pearson_correlation_coefficient(participant_Y, predicted_scores),
            'KendallTau': kendalls_tau_coefficient(participant_Y, predicted_scores)
        })
    return pd.DataFrame(evaluation_results)

def concordance_correlation_coefficient(y_true, y_pred):
    cor = np.corrcoef(y_true, y_pred)[0, 1]
    mean_true, mean_pred = np.mean(y_true), np.mean(y_pred)
    var_true, var_pred = np.var(y_true), np.var(y_pred)
    sd_true, sd_pred = np.std(y_true), np.std(y_pred)
    return 2 * cor * sd_true * sd_pred / (var_true + var_pred + (mean_true - mean_pred) ** 2)

def pearson_correlation_coefficient(y_true, y_pred):
    return pearsonr(y_true, y_pred)[0]

def kendalls_tau_coefficient(y_true, y_pred):
    return kendalltau(y_true, y_pred)[0]

input_file = os.path.join(base_dir, 'Data_Percentiles', 'ordinal_logistic_regression_percentiles.csv')

df = pd.read_csv(input_file)

df = df[df['[control]game'] == 'Heist!']
feature_cols = [col for col in df.columns if col.startswith('[general]')]

X = df[feature_cols].values
Y = df['arousal_label'].values

groups = df['[control]player_id'].values
game_name = df['[control]game'].iloc[0]

# Initialize the RankNet model
input_shape = (len(feature_cols) * 2,)
ranknet_model = RankNet(input_shape)
ranknet_model.compile(optimizer='adam', loss='binary_crossentropy')

# GroupKFold Cross-validation
group_kfold = GroupKFold(n_splits=5)

evaluation_results = []

for train_index, test_index in group_kfold.split(X, Y, groups):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    group_labels_train = groups[train_index]
    group_labels_test = groups[test_index]

    X_train_transformed, Y_train_transformed, groups_train_transformed = pairwise_transformation(X_train, Y_train, group_labels_train)
    X_test_transformed, Y_test_transformed, groups_test_transformed = pairwise_transformation(X_test, Y_test, group_labels_test)

    ranknet_model.fit(X_train_transformed, Y_train_transformed, epochs=2, verbose=1)
    
    individual_results_df = evaluate_individual_performance(ranknet_model, X_test_transformed, Y_test_transformed, groups_test_transformed, game_name)
    evaluation_results.append(individual_results_df)
    
    del X_train_transformed, Y_train_transformed
    del X_test_transformed, Y_test_transformed 
    gc.collect()

combined_results_df = pd.concat(evaluation_results, ignore_index=True)
combined_evaluation_results_file = os.path.join(base_dir, 'Evaluation', 'RankNET', 'ranknet_evaluation_results_Hesit!.csv')
combined_results_df.to_csv(combined_evaluation_results_file, index=False)

print("Individual performance evaluation results saved for Heist!")


Epoch 1/2
358089/358089 ━━━━━━━━━━━━━━━━━━━━ 2145s 6ms/step - loss: 3.1504
Epoch 2/2
358089/358089 ━━━━━━━━━━━━━━━━━━━━ 1480s 4ms/step - loss: 3.0362
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1036/1036 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
985/985 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1019/1019 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1034/1034 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
986/986 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1102/1102 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1009/1009 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1027/1027 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1015/1015 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms

- Evaluation for Shootout game

In [8]:
base_dir = 'AGAIN Ranking Algorithms'

def pairwise_transformation(X, Y, groups):
    transformed_data, labels, new_groups = [], [], []
    for i in range(len(X)):
        for j in range(i+1, len(X)):  
            if abs(Y[i] - Y[j]) > 0.1:
                transformed_data.append(np.concatenate([X[i], X[j]]))
                labels.append(1 if Y[i] > Y[j] else 0)
                new_groups.append(groups[i])

                transformed_data.append(np.concatenate([X[j], X[i]]))
                labels.append(0 if Y[i] > Y[j] else 1)
                new_groups.append(groups[j])
    return np.array(transformed_data), np.array(labels), np.array(new_groups)

class RankNet(Model):
    def __init__(self, input_shape):
        super(RankNet, self).__init__()
        self.dense1 = Dense(16, activation='relu')
        self.dropout = Dropout(0.1)
        self.dense2 = Dense(8, activation='relu')
        self.output_layer = Dense(1, activation='sigmoid')  
        self.subtract_layer = Subtract()
        
    def call(self, inputs):
        half = inputs.shape[1] // 2
        xi, xj = inputs[:, :half], inputs[:, half:]
        xi_output = self.dense2(self.dropout(self.dense1(xi)))
        xj_output = self.dense2(self.dropout(self.dense1(xj)))
        oi = self.output_layer(xi_output)
        oj = self.output_layer(xj_output)
        oij = self.subtract_layer([oi, oj])
        return oij 

def evaluate_individual_performance(model, X_test, Y_test, group_labels, game_name):
    evaluation_results = []
    for participant_id in np.unique(group_labels):
        idx = group_labels == participant_id
        participant_X, participant_Y = X_test[idx], Y_test[idx]
        predicted_scores = model.predict(participant_X).flatten()
        
        evaluation_results.append({
            'Game': game_name,
            'Participant ID': participant_id,
            'CCC': concordance_correlation_coefficient(participant_Y, predicted_scores),
            'PCC': pearson_correlation_coefficient(participant_Y, predicted_scores),
            'KendallTau': kendalls_tau_coefficient(participant_Y, predicted_scores)
        })
    return pd.DataFrame(evaluation_results)

def concordance_correlation_coefficient(y_true, y_pred):
    cor = np.corrcoef(y_true, y_pred)[0, 1]
    mean_true, mean_pred = np.mean(y_true), np.mean(y_pred)
    var_true, var_pred = np.var(y_true), np.var(y_pred)
    sd_true, sd_pred = np.std(y_true), np.std(y_pred)
    return 2 * cor * sd_true * sd_pred / (var_true + var_pred + (mean_true - mean_pred) ** 2)

def pearson_correlation_coefficient(y_true, y_pred):
    return pearsonr(y_true, y_pred)[0]

def kendalls_tau_coefficient(y_true, y_pred):
    return kendalltau(y_true, y_pred)[0]

input_file = os.path.join(base_dir, 'Data_Percentiles', 'ordinal_logistic_regression_percentiles.csv')

df = pd.read_csv(input_file)

df = df[df['[control]game'] == 'Shootout']
feature_cols = [col for col in df.columns if col.startswith('[general]')]

X = df[feature_cols].values
Y = df['arousal_label'].values

groups = df['[control]player_id'].values
game_name = df['[control]game'].iloc[0]

# Initialize the RankNet model
input_shape = (len(feature_cols) * 2,)
ranknet_model = RankNet(input_shape)
ranknet_model.compile(optimizer='adam', loss='binary_crossentropy')

# GroupKFold Cross-validation
group_kfold = GroupKFold(n_splits=5)

evaluation_results = []

for train_index, test_index in group_kfold.split(X, Y, groups):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    group_labels_train = groups[train_index]
    group_labels_test = groups[test_index]

    X_train_transformed, Y_train_transformed, groups_train_transformed = pairwise_transformation(X_train, Y_train, group_labels_train)
    X_test_transformed, Y_test_transformed, groups_test_transformed = pairwise_transformation(X_test, Y_test, group_labels_test)

    ranknet_model.fit(X_train_transformed, Y_train_transformed, epochs=2, verbose=1)
    
    individual_results_df = evaluate_individual_performance(ranknet_model, X_test_transformed, Y_test_transformed, groups_test_transformed, game_name)
    evaluation_results.append(individual_results_df)
    
    del X_train_transformed, Y_train_transformed
    del X_test_transformed, Y_test_transformed 
    gc.collect()

combined_results_df = pd.concat(evaluation_results, ignore_index=True)
combined_evaluation_results_file = os.path.join(base_dir, 'Evaluation', 'RankNET', 'ranknet_evaluation_results_Shootout.csv')
combined_results_df.to_csv(combined_evaluation_results_file, index=False)

print("Individual performance evaluation results saved for Shootout")


Epoch 1/2
330874/330874 ━━━━━━━━━━━━━━━━━━━━ 391s 1ms/step - loss: 2.7941
Epoch 2/2
330874/330874 ━━━━━━━━━━━━━━━━━━━━ 387s 1ms/step - loss: 2.5904
984/984 ━━━━━━━━━━━━━━━━━━━━ 1s 897us/step
989/989 ━━━━━━━━━━━━━━━━━━━━ 1s 811us/step
984/984 ━━━━━━━━━━━━━━━━━━━━ 1s 978us/step
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 990us/step
960/960 ━━━━━━━━━━━━━━━━━━━━ 1s 808us/step
962/962 ━━━━━━━━━━━━━━━━━━━━ 1s 848us/step
955/955 ━━━━━━━━━━━━━━━━━━━━ 1s 833us/step
966/966 ━━━━━━━━━━━━━━━━━━━━ 1s 865us/step
948/948 ━━━━━━━━━━━━━━━━━━━━ 1s 914us/step
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 1s 933us/step
935/935 ━━━━━━━━━━━━━━━━━━━━ 1s 852us/step
968/968 ━━━━━━━━━━━━━━━━━━━━ 1s 892us/step
974/974 ━━━━━━━━━━━━━━━━━━━━ 1s 903us/step
969/969 ━━━━━━━━━━━━━━━━━━━━ 1s 909us/step
974/974 ━━━━━━━━━━━━━━━━━━━━ 1s 870us/step
979/979 ━━━━━━━━━━━━━━━━━━━━ 1s 865us/step
973/973 ━━━━━━━━━━━━━━━━━━━━ 1s 836us/step
964/964 ━━━━━━━━━━━━━━━━━━━━ 1s 829us/step
988/988 ━━━━━━━━━━━━━━━━━━━━ 1s 839us/step
921/921 ━━━━━━━━━━━━━━━━━━━━ 1s 8

- Evaluation for TopDown game

In [9]:
base_dir = 'AGAIN Ranking Algorithms'

def pairwise_transformation(X, Y, groups):
    transformed_data, labels, new_groups = [], [], []
    for i in range(len(X)):
        for j in range(i+1, len(X)):  
            if abs(Y[i] - Y[j]) > 0.1:
                transformed_data.append(np.concatenate([X[i], X[j]]))
                labels.append(1 if Y[i] > Y[j] else 0)
                new_groups.append(groups[i])

                transformed_data.append(np.concatenate([X[j], X[i]]))
                labels.append(0 if Y[i] > Y[j] else 1)
                new_groups.append(groups[j])
    return np.array(transformed_data), np.array(labels), np.array(new_groups)

class RankNet(Model):
    def __init__(self, input_shape):
        super(RankNet, self).__init__()
        self.dense1 = Dense(16, activation='relu')
        self.dropout = Dropout(0.1)
        self.dense2 = Dense(8, activation='relu')
        self.output_layer = Dense(1, activation='sigmoid')  
        self.subtract_layer = Subtract()
        
    def call(self, inputs):
        half = inputs.shape[1] // 2
        xi, xj = inputs[:, :half], inputs[:, half:]
        xi_output = self.dense2(self.dropout(self.dense1(xi)))
        xj_output = self.dense2(self.dropout(self.dense1(xj)))
        oi = self.output_layer(xi_output)
        oj = self.output_layer(xj_output)
        oij = self.subtract_layer([oi, oj])
        return oij 

def evaluate_individual_performance(model, X_test, Y_test, group_labels, game_name):
    evaluation_results = []
    for participant_id in np.unique(group_labels):
        idx = group_labels == participant_id
        participant_X, participant_Y = X_test[idx], Y_test[idx]
        predicted_scores = model.predict(participant_X).flatten()
        
        evaluation_results.append({
            'Game': game_name,
            'Participant ID': participant_id,
            'CCC': concordance_correlation_coefficient(participant_Y, predicted_scores),
            'PCC': pearson_correlation_coefficient(participant_Y, predicted_scores),
            'KendallTau': kendalls_tau_coefficient(participant_Y, predicted_scores)
        })
    return pd.DataFrame(evaluation_results)

def concordance_correlation_coefficient(y_true, y_pred):
    cor = np.corrcoef(y_true, y_pred)[0, 1]
    mean_true, mean_pred = np.mean(y_true), np.mean(y_pred)
    var_true, var_pred = np.var(y_true), np.var(y_pred)
    sd_true, sd_pred = np.std(y_true), np.std(y_pred)
    return 2 * cor * sd_true * sd_pred / (var_true + var_pred + (mean_true - mean_pred) ** 2)

def pearson_correlation_coefficient(y_true, y_pred):
    return pearsonr(y_true, y_pred)[0]

def kendalls_tau_coefficient(y_true, y_pred):
    return kendalltau(y_true, y_pred)[0]

input_file = os.path.join(base_dir, 'Data_Percentiles', 'ordinal_logistic_regression_percentiles.csv')

df = pd.read_csv(input_file)

df = df[df['[control]game'] == 'TopDown']
game_name = df['[control]game'].iloc[0]
feature_cols = [col for col in df.columns if col.startswith('[general]')]

X = df[feature_cols].values
Y = df['arousal_label'].values

groups = df['[control]player_id'].values

# Initialize the RankNet model
input_shape = (len(feature_cols) * 2,)
ranknet_model = RankNet(input_shape)
ranknet_model.compile(optimizer='adam', loss='binary_crossentropy')

# GroupKFold Cross-validation
group_kfold = GroupKFold(n_splits=5)

evaluation_results = []

for train_index, test_index in group_kfold.split(X, Y, groups):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    group_labels_train = groups[train_index]
    group_labels_test = groups[test_index]

    X_train_transformed, Y_train_transformed, groups_train_transformed = pairwise_transformation(X_train, Y_train, group_labels_train)
    X_test_transformed, Y_test_transformed, groups_test_transformed = pairwise_transformation(X_test, Y_test, group_labels_test)

    ranknet_model.fit(X_train_transformed, Y_train_transformed, epochs=2, verbose=1)
    
    individual_results_df = evaluate_individual_performance(ranknet_model, X_test_transformed, Y_test_transformed, groups_test_transformed, game_name)
    evaluation_results.append(individual_results_df)
    
    del X_train_transformed, Y_train_transformed
    del X_test_transformed, Y_test_transformed
    gc.collect()

combined_results_df = pd.concat(evaluation_results, ignore_index=True)
combined_evaluation_results_file = os.path.join(base_dir, 'Evaluation', 'RankNET', 'ranknet_evaluation_results_TopDown.csv')
combined_results_df.to_csv(combined_evaluation_results_file, index=False)

print("Individual performance evaluation results saved for TopDown")


Epoch 1/2
398606/398606 ━━━━━━━━━━━━━━━━━━━━ 676s 2ms/step - loss: 2.8567
Epoch 2/2
398606/398606 ━━━━━━━━━━━━━━━━━━━━ 663s 2ms/step - loss: 2.7304
1063/1063 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
1156/1156 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
1077/1077 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1082/1082 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1072/1072 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1066/1066 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1065/1065 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1074/1074 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1082/1082 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1077/1077 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1074/1074 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1038/1038 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1127/1127 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1072/1072 ━━━━━━━━━━━━━━━━━━━━ 1s 1